In [1]:
import torchvision
from model import *
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

Loading Data

In [2]:
train_data = torchvision.datasets.CIFAR10("../P28-30_mock_project_practice/data/train", train=True, transform=torchvision.transforms.ToTensor(),download=True)
##  training data has larger dataset which is used for train
test_data = torchvision.datasets.CIFAR10("../P28-30_mock_project_practice/data/test", train=False, transform=torchvision.transforms.ToTensor(),download=True)
## smaller dataset, only for test the trained result

Files already downloaded and verified
Files already downloaded and verified


Data Length:

In [3]:
train_data_size = len(train_data)
test_data_size = len(test_data)
print("train data set length: {}".format(train_data_size))
print("test data set length: {}".format(test_data_size))

train data set length: 50000
test data set length: 10000


Load data as batch:

In [4]:
batchsize = 64
train_dataloader = DataLoader(train_data, batch_size = batchsize)
test_dataloader = DataLoader(test_data, batch_size = batchsize)

Load MODEL:

In [5]:
class CIFAR10model(nn.Module):
    def __init__(self):
        super(CIFAR10model, self).__init__()
        self.model1 = Sequential(
          Conv2d(3, 32, 5, padding=2),
          MaxPool2d(2),
          Conv2d(32,32,5, padding=2),
          MaxPool2d(2),
          Conv2d(32,64,5, padding=2),
          MaxPool2d(2),
          Flatten(),
          Linear(1024,64),             # be careful, the last step is linearing 1024 data to 64 then linear to 10
          Linear(64,10)
        )

    def forward(self, input):
        output1 = self.model1(input)
        return output1

CIFAR10Model = CIFAR10model()

if torch.cuda.is_available():                # if GPU is available -> use GPU for trainning
  CIFAR10Model = CIFAR10Model.cuda()           

Loss Function:

In [6]:
loss_fn = nn.CrossEntropyLoss()


if torch.cuda.is_available():                # if GPU is available -> use GPU for trainning
  loss_fn = loss_fn.cuda()         

Optimizer:

In [7]:
Learning_Rate = 0.01
optimizer = torch.optim.SGD(CIFAR10Model.parameters(), lr = Learning_Rate)

Trainning process:

In [8]:
total_train_step = 0
epoch = 5
writer = SummaryWriter("logs")

In [12]:
for i in range(epoch):
    print("No.{} round trainning".format(i+1))
    
    # Train
    for data in train_dataloader:
        imgs, target = data
        if torch.cuda.is_available():                # if GPU is available -> use GPU for trainning
            imgs = imgs.cuda()
            target = target.cuda()
        output = CIFAR10Model(imgs)
        loss = loss_fn(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        
        if total_train_step % 500 ==0 :
            print("trained {} times, loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
            
    # Test
    with torch.no_grad():     # torch.no_grad() = This process no need to calculate gradient
        total_accuracy = 0
        total_test_step = 0
        total_test_loss = 0
        
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():                # if GPU is available -> use GPU for trainning
                imgs = imgs.cuda()
                targets = targets.cuda()
            
            outputs = CIFAR10Model(imgs)
            loss = loss_fn(outputs, targets)
            
            total_test_loss += loss.item()
            total_test_step += 1
            
            accuracy = (outputs.argmax(1) == targets).sum()/batchsize   # Calculate the accuracy for a batch i.e. 64 images
            total_accuracy += accuracy
            
            if total_test_step % 50 ==0 :
                print("test {} times, loss: {}".format(total_test_step, total_test_loss))
                writer.add_scalar("test_loss", total_test_loss, total_test_step)
                print("Accuracy: {}".format(total_accuracy/total_test_step))
                writer.add_scalar("Accuracy", total_accuracy/total_test_step, total_test_step)

writer.close()
            

No.1 round trainning
trained 4000 times, loss: 1.4154222011566162
trained 4500 times, loss: 1.3150341510772705
test 50 times, loss: 77.9490293264389
Accuracy: 0.4362500011920929
test 100 times, loss: 156.11858093738556
Accuracy: 0.4364062547683716
test 150 times, loss: 234.2795271873474
Accuracy: 0.4345833361148834
No.2 round trainning
trained 5000 times, loss: 1.4145859479904175
test 50 times, loss: 74.32747602462769
Accuracy: 0.4556249976158142
test 100 times, loss: 148.45159327983856
Accuracy: 0.46187499165534973
test 150 times, loss: 223.0605412721634
Accuracy: 0.46041667461395264
No.3 round trainning
trained 5500 times, loss: 1.1655455827713013
trained 6000 times, loss: 1.570720911026001
test 50 times, loss: 71.30336248874664
Accuracy: 0.48656249046325684
test 100 times, loss: 142.18683099746704
Accuracy: 0.4920312464237213
test 150 times, loss: 213.97794806957245
Accuracy: 0.48885416984558105
No.4 round trainning
trained 6500 times, loss: 1.60708487033844
trained 7000 times, loss

Save Result:

In [10]:
torch.save(CIFAR10Model, "CIFAR10Model_trained.pth")

In [11]:
print(len(targets))
print(len(target))
print(len(outputs))
print(len(output))
print(output)

16
16
16
16
tensor([[-0.0790, -1.1381,  0.9373,  0.8728,  0.9242,  0.6612,  0.6203, -0.3582,
         -0.6918, -1.3039],
        [ 3.2277,  0.5355, -0.2017, -1.2475, -0.5203, -2.6708, -1.4253, -2.9033,
          4.2837,  1.1920],
        [-0.9420, -3.5167,  0.0543,  1.7124, -0.0166,  2.1941,  0.0633,  2.8640,
         -1.7983, -0.5040],
        [-0.5109, -2.2419,  1.7499,  1.8696,  0.9789,  1.8819,  1.1167,  0.5312,
         -2.7766, -1.7566],
        [-0.8496, -3.0060,  0.7435,  1.8666,  1.3408,  1.0453,  1.7055,  0.0287,
         -1.0541, -1.6217],
        [ 2.0265,  1.4208, -1.7272,  0.4955, -1.1834,  0.4695, -2.3182,  1.7397,
         -0.8777,  0.3401],
        [-0.0994, -4.2953,  2.7156,  2.6465,  0.9771,  1.9060, -0.2006,  0.6467,
         -1.1107, -2.4073],
        [ 0.4021, -2.9849,  1.2452,  3.1194, -0.2793,  3.2735, -1.4913,  1.1574,
         -1.1062, -2.1964],
        [ 0.9762, -0.1523,  0.2748, -0.7890,  0.7620, -1.1369, -0.7303,  0.9316,
          0.2289, -0.1232],
       